In [38]:
import tensorflow as tf
from numpy import savez_compressed, load
from os import listdir
from keras.models import load_model
from tensorflow import keras
import matplotlib.pyplot as plt
from PIL import Image 
import numpy as np
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import *
from keras.initializers import RandomNormal
from keras.optimizers import *
from numpy.random import randn
from numpy.random import randint
from numpy import zeros
from numpy.random import random
from numpy import vstack
import numpy as np
from keras.datasets.fashion_mnist import load_data
from numpy.random import rand 
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [39]:
def prepare_real_data():
  (trainx, trainy), (testx, testy) = load_data()
  trainx = np.expand_dims(trainx, axis=-1)
  trainx = trainx.astype("float32")
  trainx = (trainx - 127.5) / 127.5
  return trainx


In [40]:
def generate_real_samples(dataset,n_samples):
  idx = randint(0, dataset.shape[0], n_samples)
  x = dataset[idx]
  y = np.ones((n_samples,1))
  return x,y

In [41]:
def generate_latent_points(latent_dim, n_samples):
  points = randn(n_samples*latent_dim)
  points = points.reshape(n_samples,latent_dim)
  return points

In [42]:
def generate_fake_samples(generator,latent_dim, n_samples):
  x_inp = generate_latent_points(latent_dim, n_samples)
  x_out = generator.predict(x_inp)
  y = np.zeros((n_samples,1))
  return x_out, y

In [43]:
def discriminator_model(input_shape=(28,28,1)):
  model = Sequential()
  model.add(Conv2D(128,(3,3), strides=(2,2),padding="same",input_shape=input_shape))
  model.add(BatchNormalization())
  model.add(LeakyReLU(alpha=0.2))
  model.add(Conv2D(128,(3,3), strides=(2,2),padding="same",input_shape=input_shape))
  model.add(BatchNormalization())
  model.add(LeakyReLU(alpha=0.2))
  model.add(Flatten())
  model.add(Dropout(0.5))
  model.add(Dense(1,activation="sigmoid"))
  optimizer = Adam(learning_rate=0.0002,beta_1=0.5)
  model.compile(loss="binary_crossentropy",optimizer=optimizer, metrics=["accuracy"])
  return model


In [44]:
def generator_model(latent_dim):
  model = Sequential()
  nodes = 128 * 7 * 7
  model.add(Dense(nodes,input_dim = latent_dim))
  model.add(BatchNormalization())
  model.add(LeakyReLU(alpha=0.2))
  model.add(Reshape((7,7,128)))
  model.add(Conv2DTranspose(128,(4,4),strides=(2,2), padding="same"))
  model.add(BatchNormalization())
  model.add(LeakyReLU(alpha=0.2))
  model.add(Conv2DTranspose(128,(4,4),strides=(2,2), padding="same"))
  model.add(BatchNormalization())
  model.add(LeakyReLU(alpha=0.2))
  model.add(Conv2D(1,(7,7),activation="tanh", padding="same"))
  return model

In [45]:
def gan_model(generator, discriminator):
  discriminator.trainable = False
  model = Sequential()
  model.add(generator)
  model.add(discriminator)
  optimizer = Adam(learning_rate=0.0002, beta_1=0.5)
  model.compile(loss="binary_crossentropy", optimizer=optimizer)
  return model

In [46]:
def summarize_performance(epoch,generator, latent_dim, n_samples=100):
  x,_ = generate_fake_samples(generator,latent_dim, n_samples)
  x = (x+1) / 2.0
  for i in range(n_samples):
    plt.subplot(10,10,1+i)
    plt.axis("off")
    plt.imshow(x[i,:,:,0], cmap="gray_r")
  plt.savefig("generated_fig_%03d.png" % (epoch + 1))
  plt.close()

In [47]:
def train_gan(generator, discriminator, gan, dataset,latent_dim, n_epochs = 100, n_batches = 128):
  batches_per_epoch = int(dataset.shape[0] / n_batches)
  half_batch = int(n_batches/2)
  for i in range(n_epochs):
    for j in range(batches_per_epoch):
      x_real,y_real = generate_real_samples(dataset, half_batch)
      _, real_acc = discriminator.train_on_batch(x_real,y_real)
      x_fake, y_fake = generate_fake_samples(generator, latent_dim, half_batch)
      _, fake_acc = discriminator.train_on_batch(x_fake,y_fake)
      x_gan = generate_latent_points(latent_dim, n_batches)
      y_gan = np.ones((n_batches,1))
      g_loss = gan.train_on_batch(x_gan, y_gan)
      print("Epoch " + str(i) + " Batch " + str(j) + " Real_acc " + str(real_acc) + " fake acc " + str(fake_acc) + " GAN LOSS " + str(g_loss))
    if (i+1) % 5 == 0 :
      summarize_performance(i,generator, latent_dim)
  generator.save("generator_model_100.h5")


In [ ]:
latent_dim = 100
dataset = prepare_real_data()
discriminator = discriminator_model()
generator = generator_model(latent_dim)
gan_model = gan_model(generator, discriminator)
train_gan(generator, discriminator, gan_model, dataset, latent_dim)


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Epoch 0 Batch 0 Real_acc 0.75 fake acc 0.234375 GAN LOSS 0.53879404
Epoch 0 Batch 1 Real_acc 0.984375 fake acc 0.140625 GAN LOSS 0.54723966
Epoch 0 Batch 2 Real_acc 0.984375 fake acc 0.09375 GAN LOSS 0.58473
Epoch 0 Batch 3 Real_acc 1.0 fake acc 0.328125 GAN LOSS 0.65704083
Epoch 0 Batch 4 Real_acc 0.921875 fake acc 0.25 GAN LOSS 0.94287395
Epoch 0 Batch 5 Real_acc 0.90625 fake acc 0.421875 GAN LOSS 1.1562059
Epoch 0 Batch 6 Real_acc 0.921875 fake acc 0.4375 GAN LOSS 1.3857083
Epoch 0 Batch 7 Real_acc 0.984375 fake acc 0.515625 GAN LOSS 1.4451187
Epoch 0 Batch 8 Real_acc 0.953125 fake acc 0.484375 GAN LOSS 1.3518002
Epoch 0 Batch 9 Real_acc 0.953125 fake acc 0.6875 GAN LOSS 1.4510827
Epoch 0 Batch 10 Real_acc 0.984375 fake acc 0.90625 GAN LOSS 1.4713917
Epoch 0 Batch 11 Real_acc 0.953125 fake acc 0.984375 GAN LOSS 1.6121508
Epoch 0 Batch 12 Real_acc 0.921875 fake acc 0.984375 GAN LOSS 1.4132546
Epoch 0 Batch 13 Real_acc 0.96875 fake acc 0.984375 GAN LOSS 1.1473271
Epoch 0 Batch 14 Real